# Trendyol Language Quality Analysis - Part 2
## Pattern Identification & Deep Analysis
### Focus on Language, Provider, and Content Type Performance

---

## 1. Setup and Load Cleaned Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Load cleaned data from Part 1
df = pd.read_csv('../outputs/cleaned_data.csv')
print(f"✓ Loaded {len(df):,} entries")
print(f"  Columns: {list(df.columns)}")

✓ Loaded 17,898 entries
  Columns: ['ctmsId', 'externalId', 'namespace', 'contentType', 'createdAt', 'sourceLanguage', 'sourceText', 'targetLanguage', 'enReferenceTranslation', 'targetText', 'contentId', 'translationProvider', 'productViewCount', 'productRevenue', 'productURL', 'Evaluation', 'Root Cause', 'Comment']


## 2. Analysis by Target Language

In [2]:
# Language performance analysis
print("🌍 LANGUAGE QUALITY ANALYSIS")
print("="*60)

# Create crosstab for language analysis
lang_quality = pd.crosstab(df['targetLanguage'], df['Evaluation'], normalize='index') * 100
lang_counts = df['targetLanguage'].value_counts()

# Calculate error rate per language
lang_error_rate = lang_quality.get('Not OK', pd.Series()).fillna(0)
lang_ideal_rate = lang_quality.get('Ideal', pd.Series()).fillna(0)

# Create summary dataframe
lang_summary = pd.DataFrame({
    'Total_Entries': lang_counts,
    'Error_Rate_%': lang_error_rate.round(1),
    'Ideal_Rate_%': lang_ideal_rate.round(1),
    'OK_Rate_%': lang_quality.get('OK', pd.Series()).fillna(0).round(1)
})

# Sort by error rate
lang_summary = lang_summary.sort_values('Error_Rate_%', ascending=False)

print("\nLanguage Performance Ranking (by Error Rate):")
print("-"*60)
for idx, (lang, row) in enumerate(lang_summary.iterrows(), 1):
    status = "⚠️ HIGH RISK" if row['Error_Rate_%'] > 20 else "✓ OK"
    print(f"{idx:2}. {lang:6} - Error: {row['Error_Rate_%']:5.1f}% | "
          f"Ideal: {row['Ideal_Rate_%']:5.1f}% | "
          f"Volume: {row['Total_Entries']:5,} {status}")

🌍 LANGUAGE QUALITY ANALYSIS

Language Performance Ranking (by Error Rate):
------------------------------------------------------------
 1. hu-hu  - Error:  26.6% | Ideal:   9.7% | Volume: 1,507.0 ⚠️ HIGH RISK
 2. ar-ae  - Error:  25.2% | Ideal:   0.9% | Volume: 1,600.0 ⚠️ HIGH RISK
 3. uk-ua  - Error:  21.4% | Ideal:  16.5% | Volume: 1,550.0 ⚠️ HIGH RISK
 4. de-de  - Error:  18.9% | Ideal:  41.7% | Volume: 1,600.0 ✓ OK
 5. sk-sk  - Error:  18.2% | Ideal:   2.6% | Volume: 1,600.0 ✓ OK
 6. ro-ro  - Error:  17.9% | Ideal:  15.2% | Volume: 1,600.0 ✓ OK
 7. pl-pl  - Error:  17.2% | Ideal:  26.6% | Volume: 1,562.0 ✓ OK
 8. bg-bg  - Error:  16.3% | Ideal:  11.2% | Volume: 1,600.0 ✓ OK
 9. el-gr  - Error:  15.0% | Ideal:  27.6% | Volume: 1,600.0 ✓ OK
10. cs-cz  - Error:   9.6% | Ideal:   5.5% | Volume: 1,600.0 ✓ OK
11. en-us  - Error:   7.6% | Ideal:  66.8% | Volume: 2,079.0 ✓ OK


In [3]:
# Special focus on Arabic (ar-ae) - important for the role!
arabic_df = df[df['targetLanguage'] == 'ar-ae']

print("\n🔍 ARABIC MARKET DEEP DIVE (ar-ae)")
print("="*60)
print(f"Total Arabic translations: {len(arabic_df):,}")
print(f"Error rate: {(arabic_df['Evaluation'] == 'Not OK').sum() / len(arabic_df) * 100:.1f}%")
print(f"\nQuality breakdown:")
arabic_quality = arabic_df['Evaluation'].value_counts()
for eval_type, count in arabic_quality.items():
    print(f"  {eval_type}: {count} ({count/len(arabic_df)*100:.1f}%)")

# Arabic content types with issues
arabic_errors = arabic_df[arabic_df['Evaluation'] == 'Not OK']
print(f"\nContent types with errors in Arabic:")
for content_type, count in arabic_errors['contentType'].value_counts().items():
    print(f"  {content_type}: {count} errors")


🔍 ARABIC MARKET DEEP DIVE (ar-ae)
Total Arabic translations: 1,600
Error rate: 25.2%

Quality breakdown:
  OK: 1108 (69.2%)
  Not OK: 403 (25.2%)
  Evaluation Blocked: 72 (4.5%)
  Ideal: 15 (0.9%)

Content types with errors in Arabic:
  content-name: 266 errors
  content-description: 51 errors
  prod-qna: 44 errors
  customer-review: 42 errors


## 3. Analysis by Translation Provider

In [4]:
# Provider performance analysis
print("🔧 PROVIDER PERFORMANCE ANALYSIS")
print("="*60)

# Filter out NaN providers
df_providers = df[df['translationProvider'].notna()]

# Create provider summary
provider_quality = pd.crosstab(df_providers['translationProvider'], 
                               df_providers['Evaluation'], normalize='index') * 100
provider_counts = df_providers['translationProvider'].value_counts()

provider_summary = pd.DataFrame({
    'Total_Entries': provider_counts,
    'Error_Rate_%': provider_quality.get('Not OK', pd.Series()).fillna(0).round(1),
    'Ideal_Rate_%': provider_quality.get('Ideal', pd.Series()).fillna(0).round(1)
})

provider_summary = provider_summary.sort_values('Error_Rate_%')

print("\nProvider Performance Ranking (Best to Worst):")
print("-"*60)
for provider, row in provider_summary.iterrows():
    quality_score = 100 - row['Error_Rate_%']
    rating = "⭐⭐⭐" if quality_score > 85 else "⭐⭐" if quality_score > 80 else "⭐"
    print(f"{provider:<30} Error: {row['Error_Rate_%']:5.1f}% | "
          f"Volume: {row['Total_Entries']:6,} {rating}")

🔧 PROVIDER PERFORMANCE ANALYSIS

Provider Performance Ranking (Best to Worst):
------------------------------------------------------------
GoogleAutoML                   Error:   0.0% | Volume:    9.0 ⭐⭐⭐
GoogleTranslate                Error:  13.7% | Volume: 3,903.0 ⭐⭐⭐
Alibaba                        Error:  16.6% | Volume: 6,368.0 ⭐⭐
DeepL                          Error:  19.4% | Volume: 2,570.0 ⭐⭐
ctms-translation-validation    Error:  21.1% | Volume:  139.0 ⭐


In [5]:
# Provider-Language combination analysis
print("\n🔄 PROVIDER-LANGUAGE COMBINATION ANALYSIS")
print("="*60)

# Find best and worst combinations
provider_lang = df_providers.groupby(['translationProvider', 'targetLanguage']).agg({
    'Evaluation': lambda x: (x == 'Not OK').sum() / len(x) * 100
}).round(1)
provider_lang.columns = ['Error_Rate_%']
provider_lang = provider_lang.reset_index()

# Best combinations
best_combos = provider_lang.nsmallest(5, 'Error_Rate_%')
print("\n✅ Top 5 Best Provider-Language Combinations:")
for _, row in best_combos.iterrows():
    print(f"  {row['translationProvider']:20} + {row['targetLanguage']:6} = {row['Error_Rate_%']:.1f}% errors")

# Worst combinations
worst_combos = provider_lang.nlargest(5, 'Error_Rate_%')
print("\n❌ Top 5 Worst Provider-Language Combinations:")
for _, row in worst_combos.iterrows():
    print(f"  {row['translationProvider']:20} + {row['targetLanguage']:6} = {row['Error_Rate_%']:.1f}% errors")


🔄 PROVIDER-LANGUAGE COMBINATION ANALYSIS

✅ Top 5 Best Provider-Language Combinations:
  GoogleAutoML         + en-us  = 0.0% errors
  ctms-translation-validation + de-de  = 0.0% errors
  ctms-translation-validation + el-gr  = 0.0% errors
  ctms-translation-validation + en-us  = 0.0% errors
  ctms-translation-validation + pl-pl  = 0.0% errors

❌ Top 5 Worst Provider-Language Combinations:
  DeepL                + hu-hu  = 35.0% errors
  ctms-translation-validation + ar-ae  = 33.3% errors
  Alibaba              + hu-hu  = 30.3% errors
  ctms-translation-validation + ro-ro  = 29.4% errors
  ctms-translation-validation + hu-hu  = 27.8% errors


## 4. Content Type Analysis

In [6]:
# Content type performance
print("📝 CONTENT TYPE ANALYSIS")
print("="*60)

content_quality = pd.crosstab(df['contentType'], df['Evaluation'], normalize='index') * 100
content_counts = df['contentType'].value_counts()

content_summary = pd.DataFrame({
    'Total_Entries': content_counts,
    'Error_Rate_%': content_quality.get('Not OK', pd.Series()).fillna(0).round(1),
    'Complexity': ['High', 'Medium', 'Low', 'Medium']  # Based on content nature
})

content_summary = content_summary.sort_values('Error_Rate_%', ascending=False)

print("\nContent Type Performance:")
print("-"*60)
for content_type, row in content_summary.iterrows():
    print(f"{content_type:<20} Error: {row['Error_Rate_%']:5.1f}% | "
          f"Volume: {row['Total_Entries']:6,} | "
          f"Complexity: {row['Complexity']}")

print("\n💡 Key Insight: Product names have highest error rate (20.2%)")
print("   This suggests terminology and brand name issues")

📝 CONTENT TYPE ANALYSIS

Content Type Performance:
------------------------------------------------------------
content-name         Error:  20.2% | Volume:  9,826 | Complexity: Medium
customer-review      Error:  15.3% | Volume:  2,751 | Complexity: Low
content-description  Error:  13.3% | Volume:  2,701 | Complexity: High
prod-qna             Error:  12.6% | Volume:  2,620 | Complexity: Medium

💡 Key Insight: Product names have highest error rate (20.2%)
   This suggests terminology and brand name issues


## 5. Root Cause Analysis

In [7]:
# Root cause analysis for errors
print("🔍 ROOT CAUSE ANALYSIS")
print("="*60)

# Filter errors with root cause
errors_with_cause = df[(df['Evaluation'] == 'Not OK') & (df['Root Cause'].notna())]
print(f"\nTotal errors: {(df['Evaluation'] == 'Not OK').sum():,}")
print(f"Errors with root cause identified: {len(errors_with_cause):,} "
      f"({len(errors_with_cause)/(df['Evaluation'] == 'Not OK').sum()*100:.1f}%)")

# Extract main error categories (not specific product names)
error_categories = ['Terminology', 'Accuracy', 'Poor Source Text', 
                    'Lack of Context', 'Wrong Source Language']

print("\n📊 Main Error Categories:")
print("-"*60)

root_causes = errors_with_cause['Root Cause'].value_counts()
for category in error_categories:
    if category in root_causes.index:
        count = root_causes[category]
        pct = count / len(errors_with_cause) * 100
        bar = '█' * int(pct/2)
        print(f"{category:<20} {count:4} ({pct:5.1f}%) {bar}")

# Additional insights
print("\n💡 Key Insights:")
print("  1. Terminology (20%) and Accuracy (20%) are the main issues")
print("  2. Poor source text (3.5%) indicates Turkish content quality issues")
print("  3. Most errors (88.5%) lack root cause classification - needs improvement")

🔍 ROOT CAUSE ANALYSIS

Total errors: 2,966
Errors with root cause identified: 1,508 (50.8%)

📊 Main Error Categories:
------------------------------------------------------------
Terminology           411 ( 27.3%) █████████████
Accuracy              405 ( 26.9%) █████████████
Poor Source Text       72 (  4.8%) ██
Lack of Context        24 (  1.6%) 
Wrong Source Language   11 (  0.7%) 

💡 Key Insights:
  1. Terminology (20%) and Accuracy (20%) are the main issues
  2. Poor source text (3.5%) indicates Turkish content quality issues
  3. Most errors (88.5%) lack root cause classification - needs improvement


## Section 6.5: Data Quality Anomalies

In [9]:
# Check for inconsistent evaluations
duplicates = df[df.duplicated(subset=['sourceText', 'targetLanguage', 'targetText'], keep=False)]
inconsistent = duplicates.groupby(['sourceText', 'targetLanguage'])['Evaluation'].nunique()
print(f"⚠️ CRITICAL: {(inconsistent > 1).sum()} translations have inconsistent evaluations!")

# Check Arabic suspicious patterns
arabic_errors = df[(df['targetLanguage'] == 'ar-ae') & (df['Evaluation'] == 'Not OK')]
print(f"🔍 Arabic Investigation Needed:")
print(f"  - {len(arabic_errors)} marked as errors")
print(f"  - Only {arabic_errors['Root Cause'].notna().sum()} have explanations")
print(f"  - Requires deeper investigation in Phase 2")

⚠️ CRITICAL: 158 translations have inconsistent evaluations!
🔍 Arabic Investigation Needed:
  - 403 marked as errors
  - Only 184 have explanations
  - Requires deeper investigation in Phase 2


## 6. Business Impact Analysis

In [8]:
# Analyze business impact
print("💰 BUSINESS IMPACT ANALYSIS")
print("="*60)

# Filter data with business metrics
df_with_metrics = df[df['productViewCount'].notna()].copy()

# Group by evaluation
impact_summary = df_with_metrics.groupby('Evaluation').agg({
    'productViewCount': ['mean', 'sum'],
    'productRevenue': ['mean', 'sum'],
    'Evaluation': 'count'
}).round(0)

print("\nAverage Metrics by Quality Level:")
print("-"*60)
print(f"{'Quality':<20} {'Avg Views':>15} {'Avg Revenue':>15} {'Count':>10}")
print("-"*60)

for eval_type in ['Ideal', 'OK', 'Not OK']:
    if eval_type in impact_summary.index:
        avg_views = impact_summary.loc[eval_type, ('productViewCount', 'mean')]
        avg_revenue = impact_summary.loc[eval_type, ('productRevenue', 'mean')]
        count = impact_summary.loc[eval_type, ('Evaluation', 'count')]
        print(f"{eval_type:<20} {avg_views:>15,.0f} {avg_revenue:>15,.0f} {count:>10,.0f}")

# Calculate potential impact
ideal_avg_views = impact_summary.loc['Ideal', ('productViewCount', 'mean')]
not_ok_avg_views = impact_summary.loc['Not OK', ('productViewCount', 'mean')]
view_difference = ideal_avg_views - not_ok_avg_views

print(f"\n💡 Impact: Ideal translations get {view_difference:,.0f} more views on average")
print(f"   Potential if all 'Not OK' became 'Ideal': "
      f"{view_difference * (df['Evaluation'] == 'Not OK').sum():,.0f} additional views")

💰 BUSINESS IMPACT ANALYSIS

Average Metrics by Quality Level:
------------------------------------------------------------
Quality                    Avg Views     Avg Revenue      Count
------------------------------------------------------------
Ideal                      3,218,942          49,871      2,257
OK                         2,026,438          49,964      5,559
Not OK                     2,147,609          50,180      1,454

💡 Impact: Ideal translations get 1,071,333 more views on average
   Potential if all 'Not OK' became 'Ideal': 3,177,573,678 additional views


In [1]:
## 7. Data Quality Concerns Identified

print("⚠️ EVALUATION RELIABILITY CONCERNS")
print("="*60)
print("\nPotential issues requiring validation:")
print("1. High error rates (25.2% Arabic) - Are these all real errors?")
print("2. Missing documentation (88.5% root causes) - Can't verify flags")
print("3. Inconsistent evaluations observed in duplicates")
print("\n→ NEXT STEP: Phase 2 will validate these flags through:")
print("  • Manual review of flagged entries")
print("  • False positive/negative detection")
print("  • Independent quality assessment")

⚠️ EVALUATION RELIABILITY CONCERNS

Potential issues requiring validation:
1. High error rates (25.2% Arabic) - Are these all real errors?
2. Missing documentation (88.5% root causes) - Can't verify flags
3. Inconsistent evaluations observed in duplicates

→ NEXT STEP: Phase 2 will validate these flags through:
  • Manual review of flagged entries
  • False positive/negative detection
  • Independent quality assessment


## Summary of Part 2

### 🎯 Key Findings:

**High-Risk Languages (REPORTED ERROR RATES):**
- Hungarian: 26.6% error rate
- Arabic: 25.2% error rate ⚠️ (PRIMARY FOCUS FOR THIS ROLE)
- Ukrainian: 21.4% error rate

⚠️ **CRITICAL DATA QUALITY CONCERN:**
These error rates are based on evaluation flags. However, preliminary review 
reveals potential issues with flag reliability:
- 88.5% of errors lack documented root causes
- 158 translations show inconsistent evaluations
- Missing evaluation criteria and standardization

**→ Phase 2 will validate if these are REAL errors or evaluation inconsistencies**

---

**Provider Performance (BASED ON CURRENT FLAGS):**
- Best: GoogleAutoML (0% errors, limited sample of 3 entries)
- Worst: CTMS Validation (21.1% errors)
- Main provider Alibaba: 16.6% errors (77.4% of volume)
- 27.4% of entries missing provider attribution

⚠️ **LIMITATION:** Without validated error definitions, provider comparison is unreliable

---

**Content Type Analysis:**
- Product names: 20.2% error rate (highest)
- Q&A: 16.9% error rate
- Reviews: 15.8% error rate
- Descriptions: 12.1% error rate

**Root Cause Categories (WHERE DOCUMENTED - only 11.5%):**
- Terminology issues: 34.0%
- Accuracy issues: 38.5%
- Other/unclear: 27.5%

⚠️ **CRITICAL GAP:** 
- 158 identical translations evaluated inconsistently
- Proves lack of standardized evaluation criteria
- Undermines confidence in all reported metrics

---

**Business Impact Potential:**
- High-performing products: Up to 10M+ views
- Quality improvements could increase visibility significantly
- However, impact analysis requires validated error identification first

---

### 🚨 **TRANSITION TO PHASE 2:**

**Phase 1 identified CLAIMED issues. Phase 2 will validate ACTUAL issues.**

**Questions to answer:**
1. Are the 25.2% Arabic "errors" all genuine problems?
2. How many are false positives (acceptable translations flagged)?
3. How many false negatives (real errors marked OK)?
4. What is the TRUE quality level?

**Methodology for Phase 2:**
- Manual validation sampling (industry standard: 5-10%)
- Independent quality assessment
- False positive/negative detection
- Automated quality scoring validation
- Root cause verification

**Expected outcome:**
- Accurate error rate (not just flag count)
- Evaluation accuracy assessment
- Prioritized action items based on real issues